In [1]:
df = pd.read_csv('./data/act_train.csv')

dfy = pd.DataFrame(df, columns=['outcome'])
dfX = df.drop('outcome', axis=1)


df = pd.concat([dfX, dfy], axis=1)

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

df.tail()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
2197286,ppl_99994,act2_4668076,2023-06-16,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 418,1
2197287,ppl_99994,act2_4743548,2023-03-30,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1832,1
2197288,ppl_99994,act2_536973,2023-01-19,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
2197289,ppl_99994,act2_688656,2023-05-02,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 199,1
2197290,ppl_99994,act2_715089,2023-06-15,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1


In [11]:
df.tail(40)

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
2197251,ppl_99994,act2_1558572,2023-02-23,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
2197252,ppl_99994,act2_156561,2023-06-28,type 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 61,1
2197253,ppl_99994,act2_159591,2023-08-23,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
2197254,ppl_99994,act2_1689515,2023-03-02,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 143,1
2197255,ppl_99994,act2_1848008,2023-01-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
2197256,ppl_99994,act2_1865477,2023-02-01,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
2197257,ppl_99994,act2_1909193,2023-03-30,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
2197258,ppl_99994,act2_1960939,2023-02-19,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
2197259,ppl_99994,act2_1997406,2023-01-18,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
2197260,ppl_99994,act2_2023527,2023-02-01,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1


In [2]:
df['activity_category'].value_counts()

type 2    904683
type 5    490710
type 3    429408
type 4    207465
type 1    157615
type 6      4253
type 7      3157
Name: activity_category, dtype: int64

In [13]:
df_type1 = df[df['activity_category']=='type 1'][['people_id', 'activity_id', 'date', 'activity_category', 'char_1', 'char_2', 'char_3', 'char_4', 
                                                  'char_5', 'char_6', 'char_7', 'char_8', 'char_9', 'outcome']].copy()

In [5]:
df_type2 = df[df['activity_category']=='type 2'][['people_id', 'activity_id', 'date', 'char_10', 'outcome']].copy()
df_type3 = df[df['activity_category']=='type 3'][['people_id', 'activity_id', 'date', 'char_10', 'outcome']].copy()
df_type4 = df[df['activity_category']=='type 4'][['people_id', 'activity_id', 'date', 'char_10', 'outcome']].copy()
df_type5 = df[df['activity_category']=='type 5'][['people_id', 'activity_id', 'date', 'char_10', 'outcome']].copy()
df_type6 = df[df['activity_category']=='type 6'][['people_id', 'activity_id', 'date', 'char_10', 'outcome']].copy()
df_type7 = df[df['activity_category']=='type 7'][['people_id', 'activity_id', 'date', 'char_10', 'outcome']].copy()

In [30]:
df_type1.tail(10)

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,outcome
2197157,ppl_99947,act1_177448,2022-09-08,type 1,type 6,type 8,type 5,type 3,type 2,type 1,type 1,type 4,type 6,0
2197158,ppl_99947,act1_222160,2022-09-08,type 1,type 3,type 8,type 1,type 3,type 2,type 2,type 3,type 4,type 6,0
2197163,ppl_99955,act1_396718,2023-06-01,type 1,type 7,type 5,type 6,type 3,type 6,type 2,type 3,type 5,type 8,0
2197192,ppl_9997,act1_135120,2023-03-21,type 1,type 3,type 1,type 1,type 3,type 2,type 2,type 3,type 4,type 2,0
2197195,ppl_99973,act1_133169,2022-09-01,type 1,type 12,type 1,type 5,type 4,type 6,type 1,type 1,type 14,type 15,0
2197196,ppl_99973,act1_272254,2022-09-01,type 1,type 12,type 1,type 5,type 4,type 6,type 1,type 1,type 2,type 17,0
2197220,ppl_99977,act1_204921,2023-01-04,type 1,type 12,type 8,type 8,type 4,type 6,type 3,type 6,type 12,type 13,1
2197227,ppl_99978,act1_245264,2023-03-16,type 1,type 30,type 6,type 3,type 3,type 1,type 2,type 2,type 4,type 1,1
2197233,ppl_99981,act1_9232,2023-01-26,type 1,type 12,type 8,type 8,type 4,type 6,type 3,type 5,type 12,type 13,0
2197245,ppl_99994,act1_330864,2023-06-27,type 1,type 5,type 2,type 1,type 3,type 1,type 2,type 5,type 4,type 6,1


In [22]:
df_type2['char_10'].value_counts()

type 1    904683
Name: char_10, dtype: int64

In [24]:
df_type2.tail(5)

,people_id,activity_id,date,char_10,outcome
2197277,ppl_99994,act2_3534467,2023-08-17,type 1,1
2197281,ppl_99994,act2_4433480,2023-01-24,type 1,1
2197285,ppl_99994,act2_4581579,2023-01-13,type 1,1
2197288,ppl_99994,act2_536973,2023-01-19,type 1,1
2197290,ppl_99994,act2_715089,2023-06-15,type 1,1


In [25]:
df_type7['char_10'].value_counts()

type 194    3157
Name: char_10, dtype: int64

In [26]:
df_type6['char_10'].value_counts()

type 110    4253
Name: char_10, dtype: int64

In [27]:
df_type5['char_10'].value_counts()

type 452     23513
type 489     23471
type 481     18019
type 433     17282
type 450     12824
type 649     11630
type 899     11427
type 400     10569
type 464     10368
type 248      7860
type 257      7349
type 420      6719
type 600      4998
type 1058     4993
type 1069     4624
type 1251     3798
type 585      3710
type 424      3572
type 596      3546
type 473      3278
type 1097     3193
type 2252     3154
type 1946     3109
type 1550     2996
type 1141     2912
             ...  
type 4645        1
type 6120        1
type 8540        1
type 7003        1
type 5498        1
type 7692        1
type 7588        1
type 8462        1
type 8917        1
type 7699        1
type 2270        1
type 7732        1
type 4640        1
type 8442        1
type 7297        1
type 8165        1
type 7754        1
type 1877        1
type 5497        1
type 2464        1
type 8282        1
type 7614        1
type 7120        1
type 7389        1
type 7001        1
Name: char_10, Length: 2747, dt

In [28]:
df_type4['char_10'].value_counts()

type 201     6574
type 297     5145
type 143     4760
type 230     3875
type 592     3598
type 123     2555
type 479     2337
type 253     2030
type 141     1726
type 103     1554
type 299     1543
type 183     1413
type 776     1298
type 506     1203
type 6       1144
type 503     1098
type 992     1035
type 21      1018
type 313      945
type 418      896
type 759      880
type 44       880
type 69       866
type 60       851
type 184      846
             ... 
type 8657       1
type 8668       1
type 2434       1
type 6949       1
type 6966       1
type 7975       1
type 3823       1
type 4453       1
type 2955       1
type 8487       1
type 3655       1
type 3270       1
type 8013       1
type 8905       1
type 7683       1
type 3265       1
type 2599       1
type 3507       1
type 6575       1
type 8106       1
type 8249       1
type 7317       1
type 7264       1
type 2043       1
type 2625       1
Name: char_10, Length: 3315, dtype: int64

In [29]:
df_type3['char_10'].value_counts()

type 23      200408
type 2       116191
type 61       35417
type 52       19515
type 8        16112
type 3        14139
type 55        8072
type 114       2769
type 515       2337
type 237       1160
type 133        965
type 984        927
type 563        854
type 125        768
type 835        588
type 330        587
type 1953       487
type 636        482
type 107        398
type 225        367
type 737        319
type 4564       249
type 197        202
type 413        179
type 1308       160
              ...  
type 3689         1
type 7864         1
type 4598         1
type 6566         1
type 6757         1
type 4768         1
type 2384         1
type 2467         1
type 2014         1
type 8673         1
type 1234         1
type 7630         1
type 6096         1
type 4740         1
type 4767         1
type 3096         1
type 5012         1
type 4995         1
type 6749         1
type 4512         1
type 3654         1
type 8545         1
type 3192         1
type 4909         1
